In [1]:
import psycopg2 as ps
from env_vars import *
import spotipy

import spotipy
import spotipy.util as util
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler, Normalizer
import pandas as pd
from pandas.io.json import json_normalize
from flask import jsonify
from joblib import load
import pickle
import numpy as np
from flask import request
from joblib import dump
from joblib import load
import pandas as pd

from env_vars import * 

# Variables for DB inserts

In [75]:
# USERNAME = 'rileypence4' #your spotify username
# CLIENT_ID = '47c4f0d3dcaa433c9ef4ec6686e5b1f1' #set at your developer account
# CLIENT_SECRET = '65812a5fb52b4a3486f59ec672d499ff' #set at your developer account
# REDIRECT_URI = 'https://google.com/' #set at your developer account, usually "http://localhost:8000"
# SCOPE = 'user-library-read' # or else
# # ps. REDIRECT_URI is crucial here. if http://localhost:8000 is not set, or with a single '/' misplaced, it will generate a connection error.

# then pass them:

token = util.prompt_for_user_token(username = USERNAME, 
                                   scope = SCOPE, 
                                   client_id = CLIENT_ID, 
                                   client_secret = CLIENT_SECRET, 
                                   redirect_uri = REDIRECT_URI)

if token:
   sp = spotipy.Spotify(auth=token)

In [76]:
token

'BQDN5qESdvaWZ0NyIbDiloX5aarwKPrkg44xb0mwHzPNEHRqaeUI4VY_-ATXb6nYbdmsK_tfAeFxONR9R3DXb5DijzwEFJMQah-9cZx6WAGCxM3dOzVkUZn4vE-iTc5BVhLJuwMdnCg7oM_ybpGo67PbBqZDcnwsrilWMfFvBt9fqczks2IHX4Z2tr4UehkH_t8J1RlTMQ'

In [77]:
class Sound_Drip:
    
    def __init__(self, token):
        self.token = token
    
    def instantiate_sp(self,token):
        sp = spotipy.Spotify(auth=token)
        return sp

    def get_user_song_id(self,sp):
        results = sp.current_user_saved_tracks()
        genre = []
        counter = 0 
        for song_number in range(0,19):
            counter += 1 
            song_id = results['items'][song_number]['track']['id']
            artist_id = self.get_artist_id(song_id)
            genre = self.get_genres(artist_id)
            if genre != []:
                break
        return song_id

    def get_acoustical_features(self,song_id,sp):
        acoustical_features = sp.audio_features(song_id)[0]
        return acoustical_features

    def get_popularity(self, song_id):
        popularity =  sp.track(song_id)['popularity']
        return popularity

    def get_artist_id(self, song_id):
        artist = sp.track(song_id)['artists'][0]['id']
        return artist

    def get_genres(self, artist):
        genre = sp.artist(artist)['genres']
        return genre
    
    def create_feature_object(self,popularity, acoustical_features):
        popularity_dict = {'popularity': popularity}
        song_features = acoustical_features
        song_features.update(popularity_dict)
        song_features = {
    "audio_features": {
        key: song_features[key] for key in song_features.keys() & {
            'popularity',
            'acousticness',
            'danceability',
            'energy',
            'instrumentalness',
            'key',
            'liveness',
            'loudness',
            'mode',
            'speechiness',
            'tempo',
            'time_signature',
            'valence'}}}

        df = pd.DataFrame.from_dict(json_normalize(song_features["audio_features"]),orient='columns')   
        df = df.reindex(sorted(df.columns), axis=1)
        return df
    
    def get_results(self,song_features_df):
        scaler = load("./models/scalar3.joblib")
        print('Scaling data...')
        data_scaled = scaler.transform(song_features_df)
        normalizer = Normalizer()
        data_normalized = normalizer.fit_transform(data_scaled)
        print('Loading pickled model...')
        model = load('./models/model5.joblib')
        results = model.kneighbors([data_normalized][0])[1:]
        print('results returned')
        return results[0]
    
    def filter_model(self,model_results,source_genre_list): 
        #loop takes KNN results and filters by source track genres
        print("filter for genres initiated")
        genre_array = pickle.load(open("./data/genres_array_2.pkl","rb"))
        filtered_list = []
        song_list_length = 20
        for output_song_index in model_results[0][1:]:
            output_genre_list = genre_array[output_song_index]
            for output_genre in output_genre_list:
                output_genre = output_genre.strip(" ")
                for source_genre in source_genre_list:
                    source_genre = "'" + source_genre + "'"
                    if source_genre == output_genre:
                        filtered_list.append(output_song_index)
                    else:
                        continue
        if len(set(filtered_list)) > song_list_length:
            print("filter found at least 20 genre matches")
            filtered_list = set(filtered_list)
            filtered_list = list(filtered_list)[0:20]
        else:
            counter = song_list_length - len(set(filtered_list))
            print(len(set(filtered_list)))
            print(counter)
            print(f'need to add {counter} items to final song output')
            for output_song_index in model_results[1:]:
                if output_song_index not in filtered_list:
                    if counter > 0:
                        filtered_list.append(output_song_index)
                        counter -= 1
                    else:
                        break
        print("filtered list with 20 unique song indices returned")
        return filtered_list
    
    def song_id_prediction_output(self,filtered_list): 
        similar_songs = []
        print('song_id_list loading...')
        song_id_array = pickle.load(open('./data/song_id_array3.pkl', 'rb'))
        print('song_id_list loaded')
        for song_row in filtered_list:
            song_id = song_id_array[song_row]
            similar_songs.append({'similarity': [.99], 'values': song_id})
        json_dict = {"songs": similar_songs}
        print("Results returned")
        return json_dict
    
    def get_user_ids(self, sp):
        current_user_dict = sp.current_user()
        display_name = current_user_dict['display_name']
        user_id = current_user_dict['id']
        return user_id, display_name

In [78]:
token

'BQDN5qESdvaWZ0NyIbDiloX5aarwKPrkg44xb0mwHzPNEHRqaeUI4VY_-ATXb6nYbdmsK_tfAeFxONR9R3DXb5DijzwEFJMQah-9cZx6WAGCxM3dOzVkUZn4vE-iTc5BVhLJuwMdnCg7oM_ybpGo67PbBqZDcnwsrilWMfFvBt9fqczks2IHX4Z2tr4UehkH_t8J1RlTMQ'

In [79]:
sd_obj = Sound_Drip(token)

In [80]:
sp = sd_obj.instantiate_sp(token)

In [81]:
sp

In [82]:
user_id, display_name = sd_obj.get_user_ids(sp)

In [83]:
user_id, display_name

('basslaughter', 'Zhenya Warshavsky')

In [12]:
user_song_id = sd_obj.get_user_song_id(sp)

In [36]:
user_song_id

'1WbwlYiiMZGQlrFGirRnHv'

In [13]:
sd = Sound_Drip(token)
sp = sd.instantiate_sp(token)
song_id = sd.get_user_song_id(sp)
acoustical_features = sd.get_acoustical_features(song_id, sp)
popularity = sd.get_popularity(song_id)
song_features_df = sd.create_feature_object(popularity, acoustical_features)
results = sd.get_results(song_features_df)
source_genre = sd.get_genres(sd.get_artist_id(song_id))
filtered_list = sd.filter_model(results,source_genre)
sd.song_id_prediction_output(filtered_list)

/Users/user/.local/share/virtualenvs/unique-song-reqs-Bk3t-qK-/lib/python3.7/site-packages/ipykernel_launcher.py:60: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


Scaling data...
Loading pickled model...
results returned
filter for genres initiated
filter found at least 20 genre matches
filtered list with 20 unique song indices returned
song_id_list loading...
song_id_list loaded
Results returned


{'songs': [{'similarity': [0.99], 'values': '2C1FzUUtHp31ZdQsTGDM2f'},
  {'similarity': [0.99], 'values': '1NVsdk9Z0xNG2sUhZmkbD0'},
  {'similarity': [0.99], 'values': '5SwPRje7oQV5RP0saxoZp8'},
  {'similarity': [0.99], 'values': '5RrWu7ddslbnrRImeW7LOs'},
  {'similarity': [0.99], 'values': '0FQjfJB4cdkVfxZHz8RV4M'},
  {'similarity': [0.99], 'values': '6PF0WYSCH6OhGKHRHOQSxG'},
  {'similarity': [0.99], 'values': '6vEAyRqugkWBfQpccdiWFU'},
  {'similarity': [0.99], 'values': '7g7aNOC904ne2PUc14Xevb'},
  {'similarity': [0.99], 'values': '4jJwm24UagUP51wTp657Xp'},
  {'similarity': [0.99], 'values': '6roY6Z2b0Q7DH3ZM4SRoTz'},
  {'similarity': [0.99], 'values': '5wmn1zWhSSDLUmLLwYO9Rj'},
  {'similarity': [0.99], 'values': '34tFidHrETMAjsiK87WVVW'},
  {'similarity': [0.99], 'values': '0rUyA7P4nPKCRVvKhFo7XL'},
  {'similarity': [0.99], 'values': '5i2859ZJXGPOzDwdZSOMiH'},
  {'similarity': [0.99], 'values': '3QOQEObqJvCi4M2SNeko7V'},
  {'similarity': [0.99], 'values': '2raNLL2FWxWNA4veLZIuAj'},

In [17]:
final_result = sd.song_id_prediction_output(filtered_list)

song_id_list loading...
song_id_list loaded
Results returned


In [98]:
song_list = []
for song in final_result["songs"]:
    song_list.append((((song["values"]))))
    
song_list

['2C1FzUUtHp31ZdQsTGDM2f',
 '1NVsdk9Z0xNG2sUhZmkbD0',
 '5SwPRje7oQV5RP0saxoZp8',
 '5RrWu7ddslbnrRImeW7LOs',
 '0FQjfJB4cdkVfxZHz8RV4M',
 '6PF0WYSCH6OhGKHRHOQSxG',
 '6vEAyRqugkWBfQpccdiWFU',
 '7g7aNOC904ne2PUc14Xevb',
 '4jJwm24UagUP51wTp657Xp',
 '6roY6Z2b0Q7DH3ZM4SRoTz',
 '5wmn1zWhSSDLUmLLwYO9Rj',
 '34tFidHrETMAjsiK87WVVW',
 '0rUyA7P4nPKCRVvKhFo7XL',
 '5i2859ZJXGPOzDwdZSOMiH',
 '3QOQEObqJvCi4M2SNeko7V',
 '2raNLL2FWxWNA4veLZIuAj',
 '5UORiENC2iuNTFWopBTJKV',
 '4vTZpkN2Ri8Lfk2149NqWC',
 '56RfNBJGUgL1ZFCB1KEJrQ',
 '2jw0Fejobu4pxfJ1RRNT4W']

# Db Calls

In [33]:
def connect():
    conn = ps.connect(host=POSTGRES_ADDRESS,
              database=POSTGRES_DBNAME,
              user=POSTGRES_USERNAME,
              password=POSTGRES_PASSWORD,
              port=POSTGRES_PORT)
    cur = conn.cursor()
    return conn,cur

In [42]:
# def run_query():
#     try:
#         cur.execute(query)
#     except Exception as e:
#         print (e.message)
#         return reconnect()

In [43]:
# run_query()

In [35]:
query = """SELECT * FROM users;"""

cur.execute(query)
cur.fetchall()

[]

In [31]:
query = """SELECT * FROM recommendations"""

cur.execute(query)
cur.fetchall()

[]

In [37]:
f'ewfjhewjfhewjhfew'

'ewfjhewjfhewjhfew'

In [94]:
# cur.execute("""DROP TABLE users;""")
# conn.commit()

In [95]:
#deciding to remove users table as it is redundant 

# cur.execute("""CREATE TABLE users
#                 (id serial PRIMARY KEY, 
#                 display_name varchar(50),
#                 userID varchar(50)
#                 );""")
# # Commit table creation
# conn.commit()

In [32]:
# cur.execute("""CREATE TABLE recommendations
#                 (id serial PRIMARY KEY,
#                 userID varchar(50), 
#                 songID varchar(50),
#                 recDate DATE 
#                 )""")
# # Commit table creation
# conn.commit()

In [ ]:
# query = """SELECT * FROM users
#             WHERE schemaname != 'pg_catalog'
#             AND schemaname != 'information_schema';"""
# cur.execute(query)
# cur.fetchall()

In [38]:
user_id

'basslaughter'

In [39]:
song_id

'1WbwlYiiMZGQlrFGirRnHv'

In [71]:
song_list

['2C1FzUUtHp31ZdQsTGDM2f',
 '1NVsdk9Z0xNG2sUhZmkbD0',
 '5SwPRje7oQV5RP0saxoZp8',
 '5RrWu7ddslbnrRImeW7LOs',
 '0FQjfJB4cdkVfxZHz8RV4M',
 '6PF0WYSCH6OhGKHRHOQSxG',
 '6vEAyRqugkWBfQpccdiWFU',
 '7g7aNOC904ne2PUc14Xevb',
 '4jJwm24UagUP51wTp657Xp',
 '6roY6Z2b0Q7DH3ZM4SRoTz',
 '5wmn1zWhSSDLUmLLwYO9Rj',
 '34tFidHrETMAjsiK87WVVW',
 '0rUyA7P4nPKCRVvKhFo7XL',
 '5i2859ZJXGPOzDwdZSOMiH',
 '3QOQEObqJvCi4M2SNeko7V',
 '2raNLL2FWxWNA4veLZIuAj',
 '5UORiENC2iuNTFWopBTJKV',
 '4vTZpkN2Ri8Lfk2149NqWC',
 '56RfNBJGUgL1ZFCB1KEJrQ',
 '2jw0Fejobu4pxfJ1RRNT4W']

In [90]:
conn, cur = connect()

In [73]:
for song_id in song_list:
    cur.execute(
        f'INSERT INTO recommendations'
        f'(userid,songid,recdate)'
        f' VALUES (\'{user_id}\',\'{song_id}\',current_timestamp);')
conn.commit()

In [93]:
cur.execute(
        f'INSERT INTO users'
        f'(display_name,userid)'
        f' VALUES (\'{display_name}\',\'{user_id}\');')
conn.commit()

In [ ]:
insertstatement1 {songid_lis[0]}
insertstatement1 {songid_lis[1]}

In [ ]:
(f'INSERT INTO users (songid, artist, track) VALUES (\"{f["songid"]}\",\"{f["artist"]}\",\"{f["track"]}\", {f["danceability"]}, {f["energy"]}, {f["key"]}, {f["loudness"]}, {f["mode"]}, {f["speechiness"]}, {f["acousticness"]},  {f["instrumentalness"]}, {f["liveness"]}, {f["valence"]}, {f["tempo"]}, {f["duration_ms"]},{f["time_signature"]});')